In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
def my_plane(point, out, largo, alto):
    pointup, pointdown = point+out, point-out
    xs = np.array([x for x in np.arange(pointdown[0], pointup[0], (pointup[0]-pointdown[0])/largo)])
    plane = np.array([y+xs for y in np.arange(pointdown[1],pointup[1],(pointup[1]-pointdown[1])/alto)*1j])
    plane2 = tf.complex(real=plane.real, imag=plane.imag)
    return(plane2)

def my_plane_plane(point, out, pixels):
    pointup, pointdown = point+out, point-out
    xs = [x for x in np.arange(pointdown[0], pointup[0], (pointup[1]-pointdown[1])/pixels)]
    ys = [y for y in np.arange(pointdown[0], pointup[0], (pointup[1]-pointdown[1])/pixels)]
          
    result = []
    for x in xs:
        for y in ys:
            result.append([x,y])
    return(np.array(result))


@tf.function
def f_wrap(x):
    max_iter = 1000
    v = tf.cast(0,tf.float64)
    v2 = 0
    z = tf.cast(tf.complex(real=1.0,imag=1.0),tf.complex64)
    
    c = tf.cast(tf.complex(real=x[0],imag=x[1]),tf.complex64)
    
    for n in range(max_iter):
        
        r2 = tf.math.real(z)*tf.math.real(z) + tf.math.imag(z)*tf.math.imag(z)
        
        
        #if tf.cond(tf.cast(tf.greater(r2,  1000), dtype=tf.bool), lambda: 1, lambda: 0):
        v = tf.math.log(r2)/(2**n)
        v2 = 1
        #break
        
        z = (z*z) + c
        
    if v2:
        h = tf.math.log(v)/tf.math.log(tf.cast(2.0,tf.float32))
        abc = np.array([1, 1/(3**(1/2)), 1/(7*3**(1/8))])*1/np.log(2.0)
        r = 255*(( 1-tf.math.cos(h*abc[0]) )/2)
        g = 255*(( 1-tf.math.cos(h*abc[1]) )/2)
        b = 255*(( 1-tf.math.cos(h*abc[2]) )/2)
        return(tf.constant([r,g,b],dtype=tf.float64))
    else:
        return(tf.constant([0,0,0],dtype=tf.float64))
    
    
@tf.function
def mandel4(plane):
    result = tf.map_fn(f_wrap, plane)
    a = result.shape[0]**(1/2)
    return(np.reshape(result,[a,a,3.0]))


def convert(m):
    real, comples = m.real, m.imag
    return(tf.dtypes.complex(real=real,imag=comples))

m_0 = tf.complex(real=np.zeros([1000,1000]),imag=np.zeros([1000,1000]))
"""
@tf.function
def wrap(m_0,m):
    
    m_0 = tf.math.multiply(m_0, m_0) + m
    return(m_0)
    """
@tf.function
def mandelbrot(iteraciones,m_0,m):
    for x in range(iteraciones):
        m_0 = wrap(m_0,m)
        
    return(m_0)
#@tf.function


@tf.function
def mandelbrot2(iteraciones,m_0,m):
    
    result = np.zeros(m_0.shape)
    ones = np.ones(m_0.shape)
    for x in range(iteraciones):
        m_0 = wrap(m_0,m)
        
        m_0_abs = abs(m_0)
        
        idx = m_0_abs == np.inf  
        idx2 = result == 0
        idxt = idx == idx2
        result = tf.where(idxt, result, x)
        
        
        
        
    return(result)

@tf.function
def mandelbrot3(iteraciones,m_0,m,scape):
    
    result = np.zeros(m_0.shape)
    result2 = np.zeros(m_0.shape)
    ones = np.ones(m_0.shape)
    for x in range(iteraciones):
        #tf.keras.backend.clear_session()
        m_0 = wrap(m_0,m)
        
        m_0_abs = abs(m_0)
        
        
        idxt = (m_0_abs > scape) & (result == 0)
        idxtt = (m_0_abs > scape) & (result2 == 0)
        result = tf.where(idxt, tf.ones_like(result)*x, result)
        result2 = tf.where(idxtt, m_0_abs, result2)
        
    result = tf.cast(result, dtype=tf.float32) 
    result2 = tf.cast(result2,dtype=tf.float32)
    mu = result - (tf.math.log(tf.math.log(result2)))/tf.math.log(2.0000)
    mu = tf.where(tf.math.is_nan(mu), tf.zeros_like(mu), mu)
    return(mu)
@tf.function
def mandelbrotderivate(iteraciones,m_0,m):
    
    result = np.zeros(m_0.shape,dtype="float64")
    ones = np.ones(m_0.shape,dtype="float64")
    m_o_absl = []
    for x in range(iteraciones):
        m_0 = wrap(m_0,m)
        m_0_abs = abs(m_0)
        idx = m_0_abs == np.inf  
        idx2 = result == 0
        idxt = idx == idx2
        result = tf.where(idxt, result, x)
        m_o_absl.append(m_0)
    result_m = m_o_absl[iteraciones-1]-m_o_absl[iteraciones-2]   
        
        
    return(abs(result_m))
              
    
    
#@tf.function
def mandelbrot_potential(iteraciones,m_0,m,const):
    def wrap(m_0,m):
    
        m_0 = tf.math.multiply(m_0, m_0) + m
        return(m_0)
    
    result0 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result1 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result2 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    ones = np.ones(m_0.shape)
    for x in range(iteraciones):
        
        
        m_0_abs = abs(m_0)**2
        
        
        idxt = tf.greater(m_0_abs,1000000 )  & (result0 == -1) & (result1 == -1) & (result2 == -1)
        v = tf.math.log(tf.math.log(m_0_abs))-tf.cast(tf.math.log(2.0),dtype=tf.float64)*tf.cast(x+1,dtype=tf.float64)
        h = v/const

        abc = tf.constant(np.array([1, 1/(3*(2**(1/2))), 1/(7*(3**(1/8)))])*(1/np.log(2.0)), dtype=tf.float64)
        #abc = tf.constant(np.array([0.1, 0.001, 0.0001])*(1/np.log(2.0)), dtype=tf.float64)
        r = 255*(( 1-tf.math.cos(h*abc[0]) )/2)
        g = 255*(( 1-tf.math.cos(h*abc[1]) )/2)
        b = 255*(( 1-tf.math.cos(h*abc[2]) )/2)
        
        
        print(x,end = "\r")
        
        result0 = tf.where(idxt, r, result0)
        result1 = tf.where(idxt, g, result1)
        result2 = tf.where(idxt, b, result2)
        m_0 = wrap(m_0,m)
        
    result0 = tf.where(result0 == -1, 0, result0)
    result1 = tf.where(result1 == -1,  0, result1)
    result2 = tf.where(result2 == -1,  0, result2)
        
   
    return([result0, result1, result2])

def mandelbrot_potential2(iteraciones,m_0,m,const):
    def wrap(m_0,m):
    
        m_0 = tf.math.multiply(m_0, m_0) + m
        return(m_0)
    eps = 0.001
    result0 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result1 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result2 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    der = np.ones([m_0.shape[0], m_0.shape[1] ])
    ones = np.ones(m_0.shape)
    for x in range(iteraciones):
        
        der_abs = abs(der)
        m_0_abs = abs(m_0)
        idxt = tf.greater(eps, der_abs )  & (result0 == -1) & (result1 == -1) & (result2 == -1)
        v = tf.math.log(tf.math.log(m_0_abs))-tf.cast(tf.math.log(2.0),dtype=tf.float64)*tf.cast(x+1,dtype=tf.float64)
        h = v/const

        abc = tf.constant(np.array([1, 1/(3*(2**(1/2))), 1/(7*(3**(1/8)))])*(1/np.log(2.0)), dtype=tf.float64)
        #abc = tf.constant(np.array([0.1, 0.001, 0.0001])*(1/np.log(2.0)), dtype=tf.float64)
        g = x*0.1
        r = x*0.1
        b = x*0.1
        
        
        
        
        result0 = tf.where(idxt, r, result0)
        result1 = tf.where(idxt, g, result1)
        result2 = tf.where(idxt, b, result2)
        
        
        
        idxt = tf.greater(m_0_abs,4 )  & (result0 == -1) & (result1 == -1) & (result2 == -1)
        v = tf.math.log(tf.math.log(m_0_abs))+tf.cast(tf.math.log(2.0),dtype=tf.float64)*tf.cast(x+1,dtype=tf.float64)
        h = v/const

        abc = tf.constant(np.array([1, 1/(3*(2**(1/2))), 1/(7*(3**(1/8)))])*(1/np.log(2.0)), dtype=tf.float64)
        #abc = tf.constant(np.array([0.1, 0.001, 0.0001])*(1/np.log(2.0)), dtype=tf.float64)
        r = x
        g = x
        b = x
        
        
        
        
        result0 = tf.where(idxt, r, result0)
        result1 = tf.where(idxt, g, result1)
        result2 = tf.where(idxt, b, result2)
        der = der*2*m_0
        m_0 = wrap(m_0,m)
        
        
        
    result0 = tf.where(result0 == -1, 0, result0)
    result1 = tf.where(result1 == -1,  0, result1)
    result2 = tf.where(result2 == -1,  0, result2)
        
   
    return([result0, result1, result2])
def mandelbrot_potential3(iteraciones,m_0,m,const):
    def wrap(m_0,m):
    
        m_0 = tf.math.multiply(m_0, m_0) + m
        return(m_0)
    
    result0 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result1 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    result2 = np.ones([m_0.shape[0], m_0.shape[1] ])*-1
    ones = np.ones(m_0.shape)
    for x in range(iteraciones):
        
        
        m_0_abs = abs(m_0)**2
        
        
        idxt = tf.greater(m_0_abs,2000 )  & (result0 == -1) & (result1 == -1) & (result2 == -1)
        h = x - tf.math.log(tf.math.log(m_0_abs))/tf.cast(tf.math.log(2.0),dtype=tf.float64)

        abc = tf.constant(np.array([1, 1/(3*(2**(1/2))), 1/(7*(3**(1/8)))])*(1/np.log(2.0)), dtype=tf.float64)
        #abc = tf.constant(np.array([0.1, 0.001, 0.0001])*(1/np.log(2.0)), dtype=tf.float64)
        r = 255*(( 1-tf.math.cos(h*abc[0]) )/2)
        g = 255*(( 1-tf.math.cos(h*abc[1]) )/2)
        b = 255*(( 1-tf.math.cos(h*abc[2]) )/2)
        
        
        
        
        result0 = tf.where(idxt, r, result0)
        result1 = tf.where(idxt, g, result1)
        result2 = tf.where(idxt, b, result2)
        m_0 = wrap(m_0,m)
        
    result0 = tf.where(result0 == -1, 0, result0)
    result1 = tf.where(result1 == -1,  0, result1)
    result2 = tf.where(result2 == -1,  0, result2)
        
   
    return([result0, result1, result2])


In [4]:
n=2000
z = 0.0000052878
m = my_plane(np.array((-0.7435669, 0.1254023)),np.array((z*(16/9),z)),n*(16/9),n)
m_0 = tf.complex(real=np.zeros(m.shape),imag=np.zeros(m.shape))
    ###-0.77568377,0.13646737
#     -0.7435669, 0.1314023
#[0:n,0:np.round(n*(16/9)).astype(int)]
 
a = mandelbrot_potential(2000, m_0,m,const=10)

result = np.zeros([m_0.shape[0], m_0.shape[1],3 ])
result[:, :, 0] = a[2]
result[:, :, 1] = a[0]
result[:, :, 2] = a[1 ]
result = result.astype(int)
fig, ax = plt.subplots(figsize=(25, 25*(16/9)))
ax.imshow(result)
ax.axis("off")
plt.show()
#filternorm=False,resample=False, interpolation=None

ResourceExhaustedError: failed to allocate memory [Op:AddV2]